# k-arm bandit based movie recommendation system

In [2]:
import numpy as np
import pandas as pd
import random
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
class RecommendationSystem:
    def __init__(self, k=10, epsilon=0.1, initial_reward=0.0):
        self.k = k 
        self.epsilon = epsilon
        self.q_values = np.full(k, initial_reward)  
        self.action_counts = np.zeros(k)  

    def select_item(self):
        if np.random.rand() < self.epsilon:
            return np.random.randint(self.k)
        else:
            return np.argmax(self.q_values)

    def update_estimates(self, action, reward):
        self.action_counts[action] += 1
        alpha = 1 / self.action_counts[action] 
        self.q_values[action] += alpha * (reward - self.q_values[action])

    def run(self, true_rewards, steps=1000):
        rewards = []
        for step in range(steps):
            action = self.select_item()
            reward = np.random.normal(true_rewards[action], 1)
            self.update_estimates(action, reward)
            rewards.append(reward)
        return rewards

In [9]:
k = 15
epsilon = 0.1
true_rewards = np.random.normal(0, 1, k)  

In [10]:
recommender = RecommendationSystem(k=k, epsilon=epsilon)
rewards = recommender.run(true_rewards, steps=1000)

In [11]:
print("Estimated rewards for items:", recommender.q_values)
print("True rewards for items:", true_rewards)

Estimated rewards for items: [ 0.35282296  1.57206815  0.63284933  3.19346708 -0.39117545  0.96970638
  1.11953268  1.19600778  0.26623694  0.19550834 -0.87818413  0.07256211
  1.06697449  0.6053049   0.8131404 ]
True rewards for items: [-0.12298806  1.46944141  0.78836644  3.16393251  0.30763008  0.40905649
  1.12646632  0.99933533  0.37411844  0.07443958 -0.65374475  0.75409298
  1.76732288 -0.25753264  0.83635296]


In [3]:
file_path = r"C:\Users\ramya\Downloads\archive (22)\imdb_top_1000.csv"
movies_df = pd.read_csv(file_path)

In [4]:
def get_reward(movie_row):
    rating = movie_row['IMDB_Rating']
    return rating / 10  

In [5]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies_df['Series_Title']) 
similarity_matrix = cosine_similarity(tfidf_matrix)

In [6]:
epsilon = 0.1

In [7]:
def get_similar_movies(input_title, top_n=10):
    try:
        idx = movies_df[movies_df['Series_Title'].str.contains(input_title, case=False, regex=False)].index[0]
    except IndexError:
        print("Movie title not found.")
        return None

    similarity_scores = list(enumerate(similarity_matrix[idx]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similar_indices = [i[0] for i in similarity_scores[1:top_n+1]]

    return similar_indices

In [8]:
def recommend_movie(input_title):
    similar_indices = get_similar_movies(input_title)
    if not similar_indices:
        return None, None

    available_arms = len(similar_indices)
    if available_arms == 0:
        return None, None

    global arm_rewards, arm_counts
    arm_rewards = np.zeros(available_arms)
    arm_counts = np.zeros(available_arms)

    if random.random() < epsilon:
        arm = random.choice(range(available_arms))
    else:
        arm = np.argmax(arm_rewards / (arm_counts + 1e-5))

    recommended_movie = movies_df.iloc[similar_indices[arm]]
    return arm, recommended_movie

In [9]:
def update_bandit(arm, reward):
    arm_counts[arm] += 1
    arm_rewards[arm] += reward

In [10]:
input_title = "Inception"

In [11]:
for i in range(50):  
    print(f"\nRound {i + 1}")
    arm, recommended_movie = recommend_movie(input_title)
    if recommended_movie is not None:
        print(f"Recommended Movie: {recommended_movie['Series_Title']} (Rating: {recommended_movie['IMDB_Rating']})")

        reward = get_reward(recommended_movie)
        update_bandit(arm, reward)
    else:
        print("No recommendation available.")


Round 1
Recommended Movie: The Shawshank Redemption (Rating: 9.3)

Round 2
Recommended Movie: The Shawshank Redemption (Rating: 9.3)

Round 3
Recommended Movie: The Shawshank Redemption (Rating: 9.3)

Round 4
Recommended Movie: The Shawshank Redemption (Rating: 9.3)

Round 5
Recommended Movie: The Shawshank Redemption (Rating: 9.3)

Round 6
Recommended Movie: The Shawshank Redemption (Rating: 9.3)

Round 7
Recommended Movie: The Shawshank Redemption (Rating: 9.3)

Round 8
Recommended Movie: The Shawshank Redemption (Rating: 9.3)

Round 9
Recommended Movie: The Shawshank Redemption (Rating: 9.3)

Round 10
Recommended Movie: The Shawshank Redemption (Rating: 9.3)

Round 11
Recommended Movie: The Shawshank Redemption (Rating: 9.3)

Round 12
Recommended Movie: The Shawshank Redemption (Rating: 9.3)

Round 13
Recommended Movie: The Shawshank Redemption (Rating: 9.3)

Round 14
Recommended Movie: The Shawshank Redemption (Rating: 9.3)

Round 15
Recommended Movie: The Shawshank Redemption (Ra